# Putting it all together

Having completed both the symmetry functions and the forward pass, we can use the modules and see how it fits together.

In [1]:
using Distributed
@everywhere using BenchmarkTools,StaticArrays,DelimitedFiles
@everywhere using MachineLearningPotential

In [2]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
rad_function = RadialType2{Float64}(0.001,11.338,[1.,1.])
ang_func = AngularType3{Float64}(0.0001,1.0,1.0,11.338,[1.,1.,1.])
ang_func2 = AngularType3{Float64}(0.0001,-1.0,2.0,11.338,[1.,1.,1.])
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []

for row in eachrow(X)
    symmfunc = RadialType2{Float64}(row[3],row[5],[row[1],row[2]])
    push!(radsymmvec,symmfunc)
end

V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []

for element in V 
    for types in T
        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types)
        push!(angularsymmvec,symmfunc)
    end
end

total_symm_vec = vcat(radsymmvec,angularsymmvec)
f_mat = cutoff_function.(sqrt.(dis2mat),Ref(total_symm_vec[1].r_cut))

file = open("$(pwd())/symfunctions.out","r+")
truevalues = readdlm(file)
close(file)
truevals = transpose(truevalues[2:end,2:end])

file = open("$(pwd())/scaling.data")
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end
G_value_vec
testradsymmvec = []

for symmindex in eachindex(eachrow(X))
    row = X[symmindex,:]
    testradsymm = RadialType2{Float64}(row[3],row[5],[row[1],row[2]],G_value_vec[symmindex])
    push!(testradsymmvec,testradsymm)
end
test_angular_symm_vec = []
n_index = 10
for element in V
    for types in T 
        
        n_index += 1

        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types,G_value_vec[n_index])

        push!(test_angular_symm_vec,symmfunc)
    end
end
renormalised_symm_vec = vcat(testradsymmvec,test_angular_symm_vec)

renormalised_truevals = copy(truevals)
for g_index in eachindex(G_value_vec)
    renormalised_truevals[g_index,:] .-=  G_value_vec[g_index][2]
    renormalised_truevals[g_index,:] = renormalised_truevals[g_index,:] ./ (G_value_vec[g_index][1] - G_value_vec[g_index][2])
end
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]
file = open("weights.029.data","r+")
weights=readdlm(file)
close(file)
weights = vec(weights)
nnp = NeuralNetworkPotential(num_nodes,activation_functions,weights)


NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841])

# We have initialised 

positions: positions

distance matrix :   dis2mat

cutoff matrix :   f_mat

symmetryfunctions:   total_symm_vec

symmmetry matrix as given by RuNNer:   truevals

symmetry functions with renorm vals:   renormalised_symm_vec

symmetry values from runner renormalised:   renormalised_truevals

potential values:   nnp

In [3]:
function energy_calculation(positions,dist2_mat,f_mat,total_symmetry_vector,neuralnetwork)
    n_atoms = length(positions)
    g_mat = total_symm_calc(positions,dist2_mat,f_mat,total_symmetry_vector)
    return sum(forward_pass(g_mat,n_atoms,neuralnetwork))
end
function threnergy_calculation(positions,dist2_mat,f_mat,total_symmetry_vector,neuralnetwork)
    n_atoms = length(positions)
    g_mat = total_thr_symm_calc(positions,dist2_mat,f_mat,total_symmetry_vector)
    return sum(forward_pass(g_mat,n_atoms,neuralnetwork))
end

threnergy_calculation (generic function with 1 method)

In [4]:
E = energy_calculation(positions,dis2mat,f_mat,renormalised_symm_vec,nnp)

-6.216814218773105

In [5]:
@benchmark energy_calculation($positions,$dis2mat,$f_mat,$renormalised_symm_vec,$nnp)

BenchmarkTools.Trial: 136 samples with 1 evaluation.
 Range (min … max):  35.434 ms … 43.529 ms  ┊ GC (min … max): 7.79% … 7.28%
 Time  (median):     36.597 ms              ┊ GC (median):    8.74%
 Time  (mean ± σ):   36.885 ms ±  1.124 ms  ┊ GC (mean ± σ):  9.79% ± 1.83%

     █▅ ▃▆          ▁▇                                         
  ▃█▃██▇███▅▆▄▄▁▅▅▆███▅▅█▅▄▄▃▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  35.4 ms         Histogram: frequency by time        42.1 ms <

 Memory estimate: 104.21 MiB, allocs estimate: 1364576.

In [7]:
E = threnergy_calculation(positions,dis2mat,f_mat,renormalised_symm_vec,nnp)

UndefVarError: UndefVarError: total_thr_symm_calc not defined